## 크롤러 환경설정

In [1]:
import re
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import pandas as pd


options = webdriver.ChromeOptions()
options.add_argument("headless")

각자 chromedriver.exe 경로 수정

In [2]:
driver_path = 'C:/Users/chaeh/chromedriver.exe'

특수문자 및 영어등 텍스트 전처리 함수 -> clean_text

In [3]:
def clean_text(text):
    cleaned_text = re.sub('[a-zA-Z]', '', text)
    cleaned_text = re.sub('[\{\}\[\]\/?,;:|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"]',
                          '', cleaned_text)
    cleaned_text = ' '.join(cleaned_text.split())
    return cleaned_text

## 경향신문

In [4]:
def biz_khan(query,driver_path,page):
    head_list = []
    data_list = []
    current_page = 1 
    last_page =(int(page)-1)*10+1
    domain = '경향신문'
    
    while current_page < last_page:
        driver = webdriver.Chrome(executable_path=driver_path, options=options) 
        url_domain = 'https://search.naver.com/search.naver?where=news&query='
        news_domain ='&sm=tab_opt&sort=0&photo=0&field=0&pd=0&ds=&de=&docid=&related=0&mynews=1&office_type=1&office_section_code=1&news_office_checked=1032&nso=&is_sug_officeid=0' 
        url = url_domain + query +'&%2Ca%3A&start='+str(current_page) + news_domain
        driver.get(url)
        soup = BeautifulSoup(driver.page_source,'html.parser')

        # 기사 url을 list형태로 반환
        try:
            news_list = soup.select('#main_pack > section > div > div.group_news > ul')
            url_list = []
            news_list = news_list[0].find_all('li')
            for url in news_list:
                url_list.append(str(url.find_all('a')[0].get('data-url')))
        except:
            print('기사가 없습니다.')
            driver.close()
            return 0
        driver.close()
        
        for url in url_list:
            if url=='None':
                continue
            driver = webdriver.Chrome(executable_path=driver_path, options=options)  # for Windows
            time.sleep(2)
            driver.get(url)
            time.sleep(2)
            # try문 안에서 각자 언론사에 맞게 수정
            try:
                soup.find('h1',id='articleTtitle')
                soup = BeautifulSoup(driver.page_source,'html.parser')
                head_list.append(soup.find('h1',id='articleTtitle').text)
                tags = soup.select_one('#container > div.main_container > div.art_cont > div.art_body').find_all('p')
                data = [clean_text(x.text) for x in tags]
                data_list.append(' '.join(data))
            except:
                try:
                    soup = BeautifulSoup(driver.page_source,'html.parser')
                    head_list.append(soup.find('h1',id='article_title').text)
                    tags = soup.select_one('#articleBody').find_all('p')
                    data = [clean_text(x.text) for x in tags]
                    data_list.append(' '.join(data))
                except:
                    print(url)
                    print('None_type')
            driver.close()
        current_page+=10

    output = pd.DataFrame({'언론사' : [domain]*len(head_list),
                          '제목' :  head_list,
                          '내용' : data_list})
    return output

## 국민일보

In [5]:
def kmib(query,driver_path,page):
    domain = '국민일보'
    driver = webdriver.Chrome(executable_path=driver_path, options=options)
    current_page = 1
    last_page =(int(page)-1)*10+1
    
    data_list = []
    head_list = []
    
    while current_page < last_page:
        driver = webdriver.Chrome(executable_path=driver_path, options=options)
        url_domain = 'https://search.naver.com/search.naver?where=news&query='
        news_domain ='&sm=tab_opt&sort=0&photo=0&field=0&pd=0&ds=&de=&docid=&related=0&mynews=1&office_type=1&office_section_code=1&news_office_checked=1005&nso=so%3Ar%2Cp%3Aall&is_sug_officeid=0' 
        url = url_domain + query +'&%2Ca%3A&start='+str(current_page) + news_domain
        #print(url)
        driver.get(url)
        soup = BeautifulSoup(driver.page_source,'html.parser')

        # 기사 url을 list형태로 반환
        try:
            news_list = soup.select('#main_pack > section > div > div.group_news > ul')
            url_list = []
            news_list = news_list[0].find_all('li')
            for url in news_list:
                url_list.append(str(url.find_all('a')[0].get('data-url')))
        except:
            print('기사가 없습니다.')
            driver.close()
            return 0
        driver.close()
        
        for url in url_list:
            if url=='None':
                continue
            driver = webdriver.Chrome(executable_path=driver_path,options=options)  # for Windows
            time.sleep(2)
            driver.get(url)
            time.sleep(2)
            # try문 안에서 각자 언론사에 맞게 수정
            try:
                soup = BeautifulSoup(driver.page_source,'html.parser')
                head_list.append(soup.select_one('#sub > div.sub_header > div > div.nwsti > h3').text)
                tags = soup.select_one("#articleBody").text
                data=tags.strip()
                data=data.strip('\t')
                data=clean_text(data)
                data_list.append(data)
            except:
                print(url)
                print('None_type')
            driver.close()
        current_page+=10
             
    output = pd.DataFrame({'언론사' : [domain]*len(head_list),
                          '제목' :  head_list,
                          '내용' : data_list})
    return output

## 내일신문

In [6]:
def naeil(query,driver_path,page):
    domain = '내일신문'
    driver = webdriver.Chrome(executable_path=driver_path, options=options)
    current_page = 1
    last_page =(int(page)-1)*10+1
    
    data_list = []
    head_list = []
    
    while current_page < last_page:
        driver = webdriver.Chrome(executable_path=driver_path, options=options)
        url_domain = 'https://search.naver.com/search.naver?where=news&query='
        news_domain ='&sm=tab_opt&sort=0&photo=0&field=0&pd=0&ds=&de=&docid=&related=0&mynews=1&office_type=1&office_section_code=1&news_office_checked=2312&nso=&is_sug_officeid=0' 
        url = url_domain + query +'&%2Ca%3A&start='+str(current_page) + news_domain
        #print(url)
        driver.get(url)
        soup = BeautifulSoup(driver.page_source,'html.parser')

        # 기사 url을 list형태로 반환
        try:
            news_list = soup.select('#main_pack > section > div > div.group_news > ul')
            url_list = []
            news_list = news_list[0].find_all('li')
            for url in news_list:
                url_list.append(str(url.find_all('a')[0].get('data-url')))
        except:
            print('기사가 없습니다.')
            driver.close()
            return 0
        driver.close()

        for url in url_list:
            if url=='None':
                continue
            driver = webdriver.Chrome(executable_path=driver_path,options=options)  # for Windows
            time.sleep(2)
            driver.get(url)
            time.sleep(2)
            # try문 안에서 각자 언론사에 맞게 수정
            try:
                soup = BeautifulSoup(driver.page_source,'html.parser')
                head_list.append(soup.select_one('#contentArea > div.caL2 > div > div.articleArea > h3').text)
                tags = soup.select_one("#contents").text
                data=tags.strip()
                data=data.strip('\t')
                data=clean_text(data)
                data_list.append(data)
            except:
                print(url)
                print('None_type')
            driver.close()
        current_page+=10  
        
    output = pd.DataFrame({'언론사' : [domain]*len(head_list),
                          '제목' :  head_list,
                          '내용' : data_list})
    return output

## 동아일보

In [7]:
def donga(query,driver_path, page):
    domain = '동아일보'
    driver = webdriver.Chrome(executable_path=driver_path, options=options)
    
    current_page =1
    last_page =(int(page)-1)*10+1
    
    data_list =[]
    head_list=[]
    
    while current_page < last_page:
        driver =webdriver.Chrome(executable_path =driver_path, options =options)
        url_domain = 'https://search.naver.com/search.naver?where=news&query='
        news_domain ='&sm=tab_opt&sort=0&photo=0&field=0&pd=0&ds=&de=&docid=&related=0&mynews=1&office_type=1&office_section_code=1&news_office_checked=1020&nso=&is_sug_officeid=0'
        url = url_domain + query + '&%2Ca%3A&start='+str(current_page) + news_domain
        driver.get(url)
        soup = BeautifulSoup(driver.page_source,'html.parser')
 
        # 기사 url을 list형태로 반환
        try:
            news_list = soup.select('#main_pack > section > div > div.group_news > ul')
            url_list = []
            news_list = news_list[0].find_all('li')
    
            for url in news_list:
                url_list.append(str(url.find_all('a')[0].get('data-url')))
            
        except:
            print('기사가 없습니다.')
            driver.close()
            return 0
        driver.close()
    
    
        for url in url_list:
            if url=='None':
                continue
            driver = webdriver.Chrome(executable_path=driver_path,options=options)
            try:
                driver.get(url)
                time.sleep(1)
            except:
                print('link error')
                continue
        # try문 안에서 각자 언론사에 맞게 수정
            try:
                soup = BeautifulSoup(driver.page_source,'html.parser')
                head_list.append(soup.select_one('#container > div.article_title > h1').text)
                tags = soup.select_one('#content > div > div.article_txt').text
                tags=clean_text(tags)
                data_list.append(tags)
            except:
                try:
                    soup = BeautifulSoup(driver.page_source,'html.parser')
                    head_list.append(soup.select_one('#content > div.article_title > h2').text)
                    tags = soup.select_one('#ct').text
                    tags=clean_text(tags)
                    data_list.append(tags)
                except:
                    print(url)
                    print('None_type')
            driver.close()
        current_page += 10
        
    output = pd.DataFrame({'언론사' : [domain]*len(head_list),
                          '제목' :  head_list,
                          '내용' : data_list})
    return output

## 매일일보

In [8]:
def m_i(query,driver_path,page):
    domain = '매일일보'
    driver = webdriver.Chrome(executable_path=driver_path, options=options) 
    
    current_page = 1
    last_page =(int(page)-1)*10+1
    
    data_list = []
    head_list = []
    
    while current_page < last_page:
        driver = webdriver.Chrome(executable_path =driver_path, options=options)
        url_domain ='https://search.naver.com/search.naver?where=news&query='
        news_domain ='&sm=tab_opt&sort=0&photo=0&field=0&pd=0&ds=&de=&docid=&related=0&mynews=1&office_type=1&office_section_code=1&news_office_checked=2385&nso=so%3Ar%2Cp%3Aall%2Ca%3Aall&is_sug_officeid=0'
        url = url_domain + query +'&%2Ca%3A&start='+str(current_page) + news_domain
        print(url)
        driver.get(url)
        soup = BeautifulSoup(driver.page_source,'html.parser')
    
        # 기사 url을 list형태로 반환
        try:
            news_list = soup.select('#main_pack > section > div > div.group_news > ul')
            url_list = []
            news_list = news_list[0].find_all('li')
    
            for url in news_list:
                url_list.append(str(url.find_all('a')[0].get('data-url')))
    
        except: 
            print('기사가 없습니다.')
            driver.close()
            return ()
        driver.close()
    
        for url in url_list:
            if url=='None':
                continue
            driver = webdriver.Chrome(executable_path=driver_path,options=options) 
            
            try:
                driver.get(url)
                time.sleep(1)
            except:
                print('link error')
                continue
        # try문 안에서 각자 언론사에 맞게 수정
            try:
                soup = BeautifulSoup(driver.page_source,'html.parser')
                head_list.append(soup.select('#user-container > div.custom-pc.float-center.max-width-1130 > header > div > div')[0].text)
                tags = soup.find(id='article-view-content-div').find_all('p')
                data = [clean_text(x.text) for x in tags]
                data_list.append(' '.join(data))
            except:
                print(url)
                print('None_type')
            driver.close()
        current_page += 10
    output = pd.DataFrame({'언론사' : [domain]*len(head_list),
                          '제목' :  head_list,
                          '내용' : data_list})
    return output

## 문화일보

In [18]:
def munhwa(query,driver_path,page):
    domain = '문화일보'
    
    current_page = 1
    last_page =(int(page)-1)*10+1
    
    head_list = []
    data_list = []
    
    
    while current_page < last_page:
        driver = webdriver.Chrome(executable_path=driver_path, options=options)
        url_domain = 'https://search.naver.com/search.naver?where=news&query='
        news_domain = '%EB%89%B4%EC%8A%A4&sm=tab_opt&sort=0&photo=0&field=0&pd=0&ds=&de=&docid=&related=0&mynews=1&office_type=1&office_section_code=1&news_office_checked=1021&nso=&is_sug_officeid=0'
        url = url_domain + query +'&%2Ca%3A&start='+str(current_page) + news_domain
        driver.get(url)
        soup = BeautifulSoup(driver.page_source,'html.parser')
        
        # 기사 url을 list형태로 반환
        try:
            news_list = soup.select('#main_pack > section > div > div.group_news > ul')
            url_list = []
            news_list = news_list[0].find_all('li')
            
            for url in news_list:
                url_list.append(str(url.find_all('a')[0].get('data-url')))
                
        except:
            print('기사가 없습니다')
            driver.close()
            return 0
        driver.close()
        
        
        for url in url_list:
            if url=='None':
                continue
            driver = webdriver.Chrome(executable_path=driver_path,options=options)  # for Windows
            try:
                driver.get(url)
                time.sleep(1)
            
            except:
                print('link error')
                continue
                # try문 안에서 각자 언론사에 맞게 수정
                
            try:
                soup = BeautifulSoup(driver.page_source,'html.parser')
                head_list.append(soup.find('span', class_ = 'title').text)
                data = soup.select('#NewsAdContent')[0].text
                data=data.strip()
                data=data.strip('\t')
                data=clean_text(data)
                data_list.append(data)
            
            except:
                try:
                    soup = BeautifulSoup(driver.page_source,'html.parser')
                    head_list.append(soup.find('h3', id = 'articleTitle').text)
                    data = soup.select('#articleBodyContents')[0].text
                    data=data.strip()
                    data=data.strip('\t')
                    data=clean_text(data)
                    data_list.append(data)
                except:
                    print(url)
                    print("None_type")
            driver.close()
        current_page+=10
        
    output = pd.DataFrame({'언론사' : [domain]*len(head_list),
                          '제목' :  head_list,
                          '내용' : data_list})
    return output

## 서울신문

In [10]:
def seoul(query,driver_path,page):
    domain = '서울신문'
    driver = webdriver.Chrome(executable_path=driver_path, options=options)
    current_page = 1
    last_page =(int(page)-1)*10+1
    
    data_list = []
    head_list = []
    
    while current_page < last_page:
        driver = webdriver.Chrome(executable_path=driver_path, options=options)
        url_domain = 'https://search.naver.com/search.naver?where=news&query='
        news_domain ='%EB%89%B4%EC%8A%A4&sm=tab_opt&sort=0&photo=0&field=0&pd=0&ds=&de=&docid=&related=0&mynews=1&office_type=1&office_section_code=1&news_office_checked=1081&nso=&is_sug_officeid=0' 
        url = url_domain + query + '&%2Ca%3A&start='+str(current_page) + news_domain
        driver.get(url)
        soup = BeautifulSoup(driver.page_source,'html.parser')
        
        # 기사 url을 list형태로 반환
        try:
            news_list = soup.select('#main_pack > section > div > div.group_news > ul')
            url_list = []
            news_list = news_list[0].find_all('li')
            for url in news_list:
                url_list.append(str(url.find_all('a')[0].get('data-url')))
        except:
            print('기사가 없습니다.')
            driver.close()
            return 0
        driver.close()
        
        for url in url_list:
            if url=='None':
                continue
            driver = webdriver.Chrome(executable_path=driver_path,options=options)  # for Windows
            time.sleep(2)
            driver.get(url)
            time.sleep(2)
            # try문 안에서 각자 언론사에 맞게 수정
            try:
                soup = BeautifulSoup(driver.page_source,'html.parser')
                head_list.append(soup.select_one('#viewWrapDiv > div.S20_title > div.S20_article_tit > h1').text)
                tags = soup.select_one("#atic_txt1").text
                data=tags.strip()
                data=data.strip('\t')
                data=clean_text(data)
                data_list.append(data)
            except:
                print(url)
                print('None_type')
            driver.close()
        current_page+=10
             
    output = pd.DataFrame({'언론사' : [domain]*len(head_list),
                          '제목' :  head_list,
                          '내용' : data_list})
    return output

## 세계일보

In [11]:
def segye(query,driver_path,page):
    domain = '세계일보'
    driver = webdriver.Chrome(executable_path=driver_path)
        
    current_page = 1
    last_page =(int(page)-1)*10+1
    
    head_list = []
    data_list = []
    
    while current_page < last_page:
        driver = webdriver.Chrome(executable_path=driver_path, options=options)
        url_domain = 'https://search.naver.com/search.naver?where=news&query='
        news_domain ='&sm=tab_opt&sort=0&photo=0&field=0&pd=0&ds=&de=&docid=&related=0&mynews=1&office_type=1&office_section_code=1&news_office_checked=1022&nso=&is_sug_officeid=0'
        url = url_domain + query +'&%2Ca%3A&start='+str(current_page)+ news_domain
        driver.get(url)
        soup = BeautifulSoup(driver.page_source,'html.parser')

        # 기사 url을 list형태로 반환
        try:
            news_list = soup.select('#main_pack > section > div > div.group_news > ul')
            url_list = []
            news_list = news_list[0].find_all('li')
            for url in news_list:
                url_list.append(str(url.find_all('a')[0].get('data-url')))
        except:
            print('기사가 없습니다.')
            driver.close()
            return 0
        driver.close()


        for url in url_list:
            if url=='None':
                continue
            driver = webdriver.Chrome(executable_path=driver_path) 
            try:
                driver.get(url)
                time.sleep(1)
            except:
                print('link error')
                continue
                
            # try문 안에서 각자 언론사에 맞게 수정
            try:
                soup = BeautifulSoup(driver.page_source,'html.parser')
                head_list.append(soup.select('#title_sns')[0].text)
                data = soup.find('article',class_='viewBox').text
                data=data.strip()
                data=clean_text(data)
                data_list.append(data)
            except:
                print(url)
                print('None_type')
            driver.close()
        current_page+=10
        
    output = pd.DataFrame({'언론사' : [domain]*len(head_list),
                          '제목' :  head_list,
                          '내용' : data_list})
    return output

## 아시아투데이

In [12]:
def asiatoday(query,driver_path,page):
    domain = '아시아투데이'
    driver = webdriver.Chrome(executable_path=driver_path)
    
    current_page = 1 
    last_page =(int(page)-1)*10+1
    
    head_list = []
    data_list = []
    
    while current_page < last_page:
        driver = webdriver.Chrome(executable_path=driver_path, options=options)
        url_domain = 'https://search.naver.com/search.naver?where=news&query='
        news_domain ='&sm=tab_opt&sort=0&photo=0&field=0&pd=0&ds=&de=&docid=&related=0&mynews=1&office_type=1&office_section_code=1&news_office_checked=2268&nso=&is_sug_officeid=0'
        url = url_domain + query +'&%2Ca%3A&start='+str(current_page) + news_domain

        driver.get(url)
        soup = BeautifulSoup(driver.page_source,'html.parser')

        # 기사 url을 list형태로 반환
        try:
            news_list = soup.select('#main_pack > sec6tion > div > div.group_news > ul')
            url_list = []
            news_list = news_list[0].find_all('li')
            for url in news_list:
                url_list.append(str(url.find_all('a')[0].get('data-url')))
        except:
            print('기사가 없습니다.')
            driver.close()
            return 0
        driver.close()


        for url in url_list:
            if url=='None':
                continue
            driver = webdriver.Chrome(executable_path=driver_path)  # for Windows
            try:
                driver.get(url)
                time.sleep(1)
            except:
                print('link error')
                continue
            # try문 안에서 각자 언론사에 맞게 수정
            try:
                soup = BeautifulSoup(driver.page_source,'html.parser')
                head_list.append(soup.select('#section_top > div > h3')[0].text)
                data = soup.find('div',class_='news_bm').text
                data=data.strip()
                data=clean_text(data)
                data_list.append(data)
            except:
                print(url)
                print('None_type')
            driver.close()
        current_page+=10
        
    output = pd.DataFrame({'언론사' : [domain]*len(head_list),
                          '제목' :  head_list,
                          '내용' : data_list})
    return output

## 전국매일신문

In [13]:
def alldays(query,driver_path,page):
    head_list = []
    data_list = []
    current_page = 1 
    last_page =(int(page)-1)*10+1
    domain = '전국매일신문'
    
    while current_page < last_page:       
        driver = webdriver.Chrome(executable_path=driver_path, options=options) 
        url_domain = 'https://search.naver.com/search.naver?where=news&query='
        news_domain = '&sm=tab_opt&sort=0&photo=0&field=0&pd=0&ds=&de=&docid=&related=0&mynews=1&office_type=1&office_section_code=1&news_office_checked=2844&nso=&is_sug_officeid=0'
        url = url_domain + query +'&%2Ca%3A&start='+str(current_page) + news_domain
        driver.get(url)
        soup = BeautifulSoup(driver.page_source,'html.parser')

        # 기사 url을 list형태로 반환
        try:
            news_list = soup.select('#main_pack > section > div > div.group_news > ul')
            url_list = []
            news_list = news_list[0].find_all('li')
            for url in news_list:
                url_list.append(str(url.find_all('a')[0].get('data-url')))
        except:
            print('기사가 없습니다.')
            driver.close()
            return 0
        driver.close()

        for url in url_list:
            if url=='None':
                continue
            try:
                driver = webdriver.Chrome(executable_path=driver_path, options=options)  # for Windows
                driver.get(url)
                time.sleep(2)
                soup = BeautifulSoup(driver.page_source,'html.parser')
                head = clean_text(soup.find('div',class_="article-head-title").text)
                head_list.append(head)
                tags = soup.select_one('#article-view-content-div').find_all('p')
                data = [clean_text(x.text) for x in tags]
                data_list.append(' '.join(data))
            except:
                print(url)
                print("None_type")
            driver.close()
        current_page+=10
    output = pd.DataFrame({'언론사' : [domain]*len(head_list),
                           '제목' :  head_list,
                           '내용' : data_list})
    return output

## 조선일보

In [14]:
def joseon(query,driver_path,page):
    head_list = []
    data_list = []
    current_page = 1
    last_page =(int(page)-1)*10+1
    domain = '조선일보'
    
    while current_page < last_page:        
        driver = webdriver.Chrome(executable_path=driver_path, options=options)
        url_domain = 'https://search.naver.com/search.naver?where=news&query='
        news_domain = '&sm=tab_opt&sort=0&photo=0&field=0&pd=0&ds=&de=&docid=&related=0&mynews=1&office_type=1&office_section_code=1&news_office_checked=1023&nso=&is_sug_officeid=0'
        url = url_domain + query +'&%2Ca%3A&start='+str(current_page) + news_domain
        driver.get(url)
        soup = BeautifulSoup(driver.page_source,'html.parser')

        # 기사 url을 list형태로 반환
        try:
            news_list = soup.select('#main_pack > section > div > div.group_news > ul')
            url_list = []
            news_list = news_list[0].find_all('li')
            for url in news_list:
                url_list.append(str(url.find_all('a')[0].get('data-url')))
        except:
            print('기사가 없습니다.')
            driver.close()
            return 0
        driver.close()

        for url in url_list:
            if url=='None':
                continue
            try:
                driver = webdriver.Chrome(executable_path=driver_path, options=options)
                driver.get(url)
                time.sleep(2)
                soup = BeautifulSoup(driver.page_source,'html.parser')
                head = clean_text(soup.find_all("h1", class_="article-header__headline | font--primary text--black")[0].text)
                head_list.append(head)
                tags = soup.find_all('p',class_=' article-body__content article-body__content-text | text--black text font--size-sm-18 font--size-md-18 font--primary')
                data = [clean_text(x.text) for x in tags]
                data_list.append(' '.join(data))
            except:
                print("None_type")
            driver.close()
        current_page+=10
    output = pd.DataFrame({'언론사' : [domain]*len(head_list),
                           '제목' :  head_list,
                           '내용' : data_list})
    return output

## 중앙일보

In [19]:
def chungang(query,driver_path,page):
    head_list = []
    data_list = []
    current_page = 1 
    last_page =(int(page)-1)*10+1
    domain = '중앙일보'
    
    while current_page < last_page:
        driver = webdriver.Chrome(executable_path=driver_path, options=options)  
        url_domain = 'https://search.naver.com/search.naver?where=news&query='
        news_domain = '&sm=tab_opt&sort=0&photo=0&field=0&pd=0&ds=&de=&docid=&related=0&mynews=1&office_type=1&office_section_code=1&news_office_checked=1025&nso=&is_sug_officeid=0'
        url = url_domain + query +'&%2Ca%3A&start='+str(current_page) + news_domain
        driver.get(url)
        soup = BeautifulSoup(driver.page_source,'html.parser')

        # 기사 url을 list형태로 반환
        try:
            news_list = soup.select('#main_pack > section > div > div.group_news > ul')
            url_list = []
            news_list = news_list[0].find_all('li')
            for url in news_list:
                url_list.append(str(url.find_all('a')[0].get('data-url')))
        except:
            print('기사가 없습니다.')
            driver.close()
            return 0
        driver.close()

        for url in url_list:
            if url=='None':
                continue
            try:
                driver = webdriver.Chrome(executable_path=driver_path, options=options)  # for Windows
                driver.get(url)
                time.sleep(2)
                soup = BeautifulSoup(driver.page_source,'html.parser')
                head = clean_text(soup.select('#article_title')[0].text)
                head_list.append(head)
                text = soup.find('div', id='article_body').text
                text = clean_text(text)
                data_list.append(text)
            except:
                print(url)
                print("None_type")
            driver.close()
        current_page+=10
    output = pd.DataFrame({'언론사' : [domain]*len(head_list),
                           '제목' :  head_list,
                           '내용' : data_list})
    return output

## 천지일보

In [20]:
def newscj(query,driver_path,page):
    head_list = []
    data_list = []
    current_page = 1
    last_page =(int(page)-1)*10+1
    domain = '천지일보'
    
    while current_page < last_page:
        driver = webdriver.Chrome(executable_path=driver_path, options=options) 
        url_domain = 'https://search.naver.com/search.naver?where=news&query='
        news_domain ='&sm=tab_opt&sort=0&photo=0&field=0&pd=0&ds=&de=&docid=&related=0&mynews=1&office_type=1&office_section_code=1&news_office_checked=2041&nso=&is_sug_officeid=0' 
        url = url_domain + query +'&%2Ca%3A&start='+str(current_page) + news_domain
        driver.get(url)
        soup = BeautifulSoup(driver.page_source,'html.parser')

        # 기사 url을 list형태로 반환
        try:
            news_list = soup.select('#main_pack > section > div > div.group_news > ul')
            url_list = []
            news_list = news_list[0].find_all('li')
            for url in news_list:
                url_list.append(str(url.find_all('a')[0].get('data-url')))
        except:
            print('기사가 없습니다.')
            driver.close()
            return 0
        driver.close()

        for url in url_list:
            if url=='None':
                continue
            try:
                driver = webdriver.Chrome(executable_path=driver_path, options=options) 
                driver.get(url)
                time.sleep(2)
                soup = BeautifulSoup(driver.page_source,'html.parser')
                head = clean_text(soup.find('div',class_="article-head-title").text)
                head_list.append(head)
                tags = soup.select_one('#article-view-content-div').find_all('p')
                data = [clean_text(x.text) for x in tags]
                data_list.append(' '.join(data))
            except:
                print(url)
                print("None_type")
            driver.close()
        current_page+=10
    output = pd.DataFrame({'언론사' : [domain]*len(head_list),
                           '제목' :  head_list,
                           '내용' : data_list})
    return output

## 한겨례

In [21]:
def hani(query,driver_path,page):
    head_list = []
    data_list = []
    current_page = 1 
    last_page =(int(page)-1)*10+1
    domain = '한겨례'
    
    while current_page < last_page:
        driver = webdriver.Chrome(executable_path=driver_path, options=options) 
        url_domain = 'https://search.naver.com/search.naver?where=news&query='
        news_domain ='&sm=tab_opt&sort=0&photo=0&field=0&pd=0&ds=&de=&docid=&related=0&mynews=1&office_type=1&office_section_code=1&news_office_checked=1028&nso=&is_sug_officeid=0' 
        url = url_domain + query +'&%2Ca%3A&start='+str(current_page) + news_domain
        driver.get(url)
        soup = BeautifulSoup(driver.page_source,'html.parser')

        # 기사 url을 list형태로 반환
        try:
            news_list = soup.select('#main_pack > section > div > div.group_news > ul')
            url_list = []
            news_list = news_list[0].find_all('li')
            for url in news_list:
                url_list.append(str(url.find_all('a')[0].get('data-url')))
        except:
            print('기사가 없습니다.')
            driver.close()
            return 0
        driver.close()

        for url in url_list:
            if url=='None':
                continue
            try:
                driver = webdriver.Chrome(executable_path=driver_path, options=options)  
                driver.get(url)
                time.sleep(2)
                soup = BeautifulSoup(driver.page_source,'html.parser')
                head = clean_text(soup.find('span', class_='title').text)
                head_list.append(head)
                tags = soup.select_one('#a-left-scroll-in > div.article-text > div').find_all('div', class_='text')
                data = [clean_text(x.text) for x in tags]
                data_list.append(' '.join(data))
            except:
                print("None_type")
            driver.close()
        current_page+=10
    output = pd.DataFrame({'언론사' : [domain]*len(head_list),
                           '제목' :  head_list,
                           '내용' : data_list})
    return output

## 한국일보

In [22]:
def hankook(query,driver_path,page):
    domain = '한국일보'
    driver = webdriver.Chrome(executable_path=driver_path, options=options)  # for Windows
    
    current_page = 3
    last_page =(int(page)-1)*10+1
    
    head_list = []
    data_list = []
    
    
    while current_page < last_page:
        driver = webdriver.Chrome(executable_path = driver_path,options=options)
        url_domain = 'https://search.naver.com/search.naver?where=news&query='
        news_domain = '%EB%89%B4%EC%8A%A4&sm=tab_opt&sort=0&photo=0&field=0&pd=0&ds=&de=&docid=&related=0&mynews=1&office_type=1&office_section_code=1&news_office_checked=1469&nso=&is_sug_officeid=0'
        url = url_domain + query +'&%2Ca%3A&start='+str(current_page) + news_domain
        driver.get(url)
        soup = BeautifulSoup(driver.page_source,'html.parser')
        
        # 기사 url을 list형태로 반환
        try:
            news_list = soup.select('#main_pack > section > div > div.group_news > ul')
            url_list = []
            news_list = news_list[0].find_all('li')
            for url in news_list:
                url_list.append(str(url.find_all('a')[0].get('data-url')))
        except:
            print('기사가 없습니다.')
            driver.close()
            return 0
        driver.close()

        for url in url_list:
            if url=='None':
                continue
            driver = webdriver.Chrome(executable_path=driver_path, options=options)  # for Windows
            driver.get(url)
            time.sleep(2)
            # try문 안에서 각자 언론사에 맞게 수정
            try:
                soup = BeautifulSoup(driver.page_source,'html.parser')
                head_list.append(clean_text(soup.find('h2').text))
                tags = soup.select_one('body > div.wrap > div.container.end.end-uni > div.end-body > div > div.col-main.read').find_all('p')
                data = [clean_text(x.text) for x in tags]
                if data=='' or data==None or data==' ':
                    data='null'
                data_list.append(data)
            except:
                print(url)
                print('None_type')
            driver.close()
        current_page+=10
        
    output = pd.DataFrame({'언론사' : [domain]*len(head_list),
                          '제목' :  head_list,
                          '내용' : data_list})
    return output

## 원하는 keyword와 페이지 수 설정

In [23]:
query = '친환경'
max_page=5

In [24]:
biz_df = biz_khan(query,driver_path,max_page)

In [25]:
kmib_df = kmib(query,driver_path,max_page)

In [26]:
naeil_df = naeil(query,driver_path,max_page)

In [27]:
donga_df = donga(query,driver_path,max_page)

In [28]:
mi_df = m_i(query,driver_path,max_page)

https://search.naver.com/search.naver?where=news&query=친환경&%2Ca%3A&start=1&sm=tab_opt&sort=0&photo=0&field=0&pd=0&ds=&de=&docid=&related=0&mynews=1&office_type=1&office_section_code=1&news_office_checked=2385&nso=so%3Ar%2Cp%3Aall%2Ca%3Aall&is_sug_officeid=0
https://search.naver.com/search.naver?where=news&query=친환경&%2Ca%3A&start=11&sm=tab_opt&sort=0&photo=0&field=0&pd=0&ds=&de=&docid=&related=0&mynews=1&office_type=1&office_section_code=1&news_office_checked=2385&nso=so%3Ar%2Cp%3Aall%2Ca%3Aall&is_sug_officeid=0
https://search.naver.com/search.naver?where=news&query=친환경&%2Ca%3A&start=21&sm=tab_opt&sort=0&photo=0&field=0&pd=0&ds=&de=&docid=&related=0&mynews=1&office_type=1&office_section_code=1&news_office_checked=2385&nso=so%3Ar%2Cp%3Aall%2Ca%3Aall&is_sug_officeid=0
https://search.naver.com/search.naver?where=news&query=친환경&%2Ca%3A&start=31&sm=tab_opt&sort=0&photo=0&field=0&pd=0&ds=&de=&docid=&related=0&mynews=1&office_type=1&office_section_code=1&news_office_checked=2385&nso=so%3Ar%2Cp

In [29]:
munhwa_df = munhwa(query,driver_path,max_page)

In [30]:
seoul_df = seoul(query,driver_path,max_page)

https://www.seoul.co.kr/news/newsView.php?id=20210811014015&wlog_tag3=naver
None_type
https://nownews.seoul.co.kr/news/newsView.php?id=20210808601001&wlog_tag3=naver
None_type
https://www.seoul.co.kr/news/newsView.php?id=20210806011012&wlog_tag3=naver
None_type
https://biz.seoul.co.kr/news/newsView.php?id=20210729500119&wlog_tag3=naver
None_type
https://www.seoul.co.kr/news/newsView.php?id=20210723013001&wlog_tag3=naver
None_type
https://www.seoul.co.kr/news/newsView.php?id=20210714500172&wlog_tag3=naver
None_type
https://www.seoul.co.kr/news/newsView.php?id=20210709016001&wlog_tag3=naver
None_type


In [31]:
segye_df = segye(query,driver_path,max_page)

In [32]:
asia_df = asiatoday(query,driver_path,max_page)

기사가 없습니다.


In [33]:
alldays_df = alldays(query, driver_path,max_page)

In [34]:
joseon_df = joseon(query, driver_path,max_page)

In [35]:
chungang_df = chungang(query, driver_path,max_page)

https://news.joins.com/article/olink/23725949
None_type
https://news.joins.com/article/olink/23724886
None_type
https://news.joins.com/article/olink/23725619
None_type
https://news.joins.com/article/olink/23723464
None_type
https://news.joins.com/article/olink/23721749
None_type
https://news.joins.com/article/olink/23721753
None_type
https://news.joins.com/article/olink/23717222
None_type
https://news.joins.com/article/olink/23712930
None_type
https://news.joins.com/article/olink/23707806
None_type
https://news.joins.com/article/olink/23709156
None_type
https://news.joins.com/article/olink/23726820
None_type
https://news.joins.com/article/olink/23725942
None_type
https://news.joins.com/article/olink/23726622
None_type
https://news.joins.com/article/olink/23726087
None_type
https://news.joins.com/article/olink/23722741
None_type
https://news.joins.com/article/olink/23725201
None_type
https://news.joins.com/article/olink/23723875
None_type
https://news.joins.com/article/olink/23726563
No

In [36]:
cj_df = newscj(query,driver_path,max_page)

In [37]:
hani_df = hani(query,driver_path,max_page)

None_type
None_type
None_type
None_type
None_type
None_type


In [38]:
hk_df = hankook(query,driver_path,max_page)

### 네이버 뉴스 모든 일간지 이름 list (dataframe 합칠때 None인 데이터들은 구별해주기 위한 작업)

In [49]:
df_list=[biz_df, kmib_df, naeil_df, donga_df, mi_df, munhwa_df, seoul_df, segye_df, asia_df, alldays_df ,joseon_df, chungang_df, cj_df, hani_df, hk_df]

In [50]:
names=[]
for name in df_list:
    if type(name)!=int:
        names.append(name)

In [51]:
news_df=pd.concat(names)
news_df.drop_duplicates(['제목'],inplace=True,ignore_index=True)

In [52]:
news_df

,언론사,제목,내용
0,경향신문,"LG화학, 2조6000억 투자해 친환경 소재 육성한다",화학 대산공장 전경. 화학 제공. 화학이 친환경 소재 사업을 육성하기 위해 대규모 ...
1,경향신문,친환경차 패권 의지 드러낸 미국···한국차 미·중·EU ‘신 경제블록’ 진입 모색해야,조 바이든 미국 대통령이 지난 5일현지시간 미국 워싱턴 백악관에서 열린 무공해차 홍...
2,경향신문,중소형 선박에 LPG엔진을···“친환경 시장 주도” 부산 전국 첫 실증 착수,부산시가 중·소형선박 엔진을 액화석유가스 엔진으로 전환하기 위한 실증작업에 나선다....
3,경향신문,"비정유·친환경 미래 먹거리 찾아나서는 정유업계, ‘탈석유’ 잰걸음",현대오일뱅크는 2030년까지 친환경미래사업의 영업이익 비중을 70까지 확대하기로 했...
4,경향신문,화학업계의 친환경 행보? '플라스틱 생산량부터 줄여야',롯데케미칼 소재 제품 포장백. 롯데케미칼 제공. 과거에는 환경오염의 주범으로 꼽혔던...
...,...,...,...
501,한국일보,실천 기업 우대 친환경기업우대론 출시,[농협은행은행장 권준학은 지난 3월 그린뉴딜 정책방향에 맞춰 친환경 경영 우수기업에...
502,한국일보,팩 친환경 포장재 개발 · · · 제품 전체에 적용,[그룹회장 허영인이 기업의 지속 가능한 성장을 추구하는 환경 사회 지배구조 경영 강...
503,한국일보,친환경 최고층 모범··· 롯데월드타워 경영 강화,[롯데월드타워는 설계 단계에서부터 친환경 에너지 생산과 효율적인 관리를 목표로 만들...
504,한국일보,年 2회 호텔 친환경 심사··· 농가 상생 프로젝트,[지난 7일 국내 호텔업계 맏형으로 손꼽히는 롯데호텔이 지속가능 경영 강화를 위해 ...


In [53]:
news_df=news_df.drop(['제목'],axis=1)

In [54]:
news_df=news_df.drop(['언론사'],axis=1)

In [55]:
news_df

,내용
0,화학 대산공장 전경. 화학 제공. 화학이 친환경 소재 사업을 육성하기 위해 대규모 ...
1,조 바이든 미국 대통령이 지난 5일현지시간 미국 워싱턴 백악관에서 열린 무공해차 홍...
2,부산시가 중·소형선박 엔진을 액화석유가스 엔진으로 전환하기 위한 실증작업에 나선다....
3,현대오일뱅크는 2030년까지 친환경미래사업의 영업이익 비중을 70까지 확대하기로 했...
4,롯데케미칼 소재 제품 포장백. 롯데케미칼 제공. 과거에는 환경오염의 주범으로 꼽혔던...
...,...
501,[농협은행은행장 권준학은 지난 3월 그린뉴딜 정책방향에 맞춰 친환경 경영 우수기업에...
502,[그룹회장 허영인이 기업의 지속 가능한 성장을 추구하는 환경 사회 지배구조 경영 강...
503,[롯데월드타워는 설계 단계에서부터 친환경 에너지 생산과 효율적인 관리를 목표로 만들...
504,[지난 7일 국내 호텔업계 맏형으로 손꼽히는 롯데호텔이 지속가능 경영 강화를 위해 ...


In [56]:
def clean_text(text):
    cleaned_text = re.sub('[^ ㄱ-ㅣ가-힣]+','.',text) # 한국어, 스페이스바, .를 제외한 모든 문자 제거
    cleaned_text = ' '.join(cleaned_text.split()) # 너무 많은 공백이 생기므로 공백 1개로
    return cleaned_text

In [57]:
news_df['내용']=news_df['내용'].apply(lambda x:clean_text(str(x)))

In [58]:
news_df.to_csv('친환경.csv', encoding='CP949')